In [3]:
#notebook to download the csv of edges and nodes of a given network
import os
import requests 
import time
import string
import networkx as nx
import itertools
import networkx as nx
import pandas as pd
import json
import re
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords

from nltk.corpus import wordnet as wn #importing it
from nltk.stem.wordnet import WordNetLemmatizer #importing wordnet lemmatizer
from nltk import pos_tag #part-of-speech-tagger
from collections import defaultdict #defaultdict returns default value for non-existant keys you try to  access based on the function you passed in the constructor
from google.colab import files

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
def extract_text(df):       #extract the text from the tweets and RT
                            #works ONLY on .csv file
    list_strings = []
    for index in range(len(df)):
        #print(index)
        text = df.loc[index]['text']                          #if it is nor trucated nor a RT  i take "text"
        string = -1
        if (df.loc[index,"truncated"] == True):                 #if it is trucated I take "extended_tweet"
            string = df.loc[index,"extended_tweet"]
        if type(df.loc[index,"retweeted_status"]) != float:     #if it is a RT I take retweeted_status
            string = df.loc[index,"retweeted_status"]
        if type(string) == str :
            if(re.search('full_text\':(.+?)https',string) != None):     #if I find "full_text"
                s = re.search('full_text\':(.+?)https',string).group(1)
            if(re.search('text\':(.+?)https',string)!= None):
                s = re.search('text\':(.+?)https',string).group(1)
            else:  #here then one should remove the http
                if (re.search('full_text\':\s\'(.+?)\'',string)!=None) :
                    s=re.search('full_text\':\s\'(.+?)\'',string).group(1)  
                if (re.search('\{\\\'full_text\\\':\s(.+?)\"',string)!=None) : 
                    re.search('\{\\\'full_text\\\':\s(.+?)\"',string).group(1)
            
            list_strings.append(s)
            #print(s)         
        else:
            list_strings.append(text)
            print(text)
      

    return list_strings

In [6]:
# Cleaning, lemmatising and pos tagging tweets

nltk.download('words')
WORDS = set(nltk.corpus.words.words()) #the last two lines serve to download the corpus of standard English language words
nltk.download('stopwords') #downloading stopwords
STOP_WORDS = set(nltk.corpus.stopwords.words("english")) #taking the stop words from English language
nltk.download('wordnet') #downloading wordnet
nltk.download('averaged_perceptron_tagger') #downloading tagger
tag_map = defaultdict(lambda : wn.NOUN) #here we define that wn.NOUN is the default value for the dict
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

def lemma_pos_cleaner(tweet):

    tweet = re.sub("@[A-Za-z0-9]+","",tweet) # remove mentions
    tweet = re.sub("#[A-Za-z0-9]+", "",tweet) # remove hashtags
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) # remove http links
    tweet = " ".join(tweet.split())
    tweet = str.lower(tweet) #to lowercase 
    tweet = re.sub("'"," ",tweet) # remove aphostrophe

    #basically we use pos_tag function on tokens that we get by applying wordpunct tokenization
    #to tweet (it separates all the words and symbols)
    #then we pass the token along with it's wordnet pos value that we get from the tag_map dictionary (noun, adjective, verb or adverb) to the lemma function (the WordNetLemmatizer())
    lemma_function = WordNetLemmatizer()
    tweet = " ".join(lemma_function.lemmatize(token, tag_map[tag[0]]) for token, tag in nltk.pos_tag(nltk.wordpunct_tokenize(tweet))) #lemmatize
  

    # francesco: I removed also all 2 letters words and added specific words, words that appears frequently but are discarded because they are not in the english language
    SPECIFIC_WORDS = ['virus', 'coronavirus', 'covid19', 'covid', 'trump', 'hubei', 'beijing', 'xinjiang', 'jinping', 'korea', 'xinhua', 'india', 'taiwan','johnson','singapore', 'africa', 'japanese', 'france', 'asian', 'australia', 'french', 'asia', 'leishenshan', 'british', 'qingdao', 'fauci', 'america',  'california', 'sichuan', 'malaysia', 'huawei','thailand', 'shandong', 'italy', 'philippines', 'germany', 'facebook', 'african', 'shenzhen', 'tokyo', 'russian','uygur', '5g', 'pompeo', 'vietnam', 'australian', 'cambodia', 'zhejiang', 'yunnan', 'guangdong', 'korean', 'iran', 'washington']
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) if (w in WORDS or w in SPECIFIC_WORDS) and len(w)>2 and w not in STOP_WORDS ) #remove stop words
   
    return tweet

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [7]:
def frequency_dictionary(df):
  unique_words = {}

  for row in df:
    for word in row.split():
      #if the word is encountered for the first time add to dict as key and set its value to 0
      unique_words.setdefault(word,0)
      #increase the value (i.e the count) of the word by 1 every time it is encountered
      unique_words[word] += 1

  return unique_words

In [8]:
period = ''  # '', '_JanFeb2020', '_MarchApril2021', '_SeptOct2020'

In [9]:
China = pd.read_csv('/content/China'+period+'.csv')
USA = pd.read_csv('/content/USA'+period+'.csv')
China_USA = pd.read_csv('/content/China&USA'+period+'.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (34,35,38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
# number of tweets:
print('China: ', len(China))
print('USA: ', len(USA))
print('China&USA: ', len(China_USA))

China:  5080
USA:  14553
China&USA:  19633


In [11]:
text_China = extract_text(China)
text_USA = extract_text(USA)
text_China_USA = extract_text(China_USA)

Output streaming troncato alle ultime 5000 righe.
Spain's far-right Vox party to challenge Madrid COVID curbs in court https://t.co/jvUxujQDT2 https://t.co/OP9LxQOq4Z
Northern England mayors call for bigger financial support ahead of new COVID-19 rules https://t.co/9hPn7IeWeO https://t.co/DP3XzucZzU
U.S. COVID-19 cases hit two-month high, 10 states report record increases https://t.co/kiN1kgbKNh https://t.co/W2DcTxnzCJ
Nurses suffer burn-out, psychological distress in COVID fight - association https://t.co/fzBcuXvxkw https://t.co/ltpEOoRM9b
Face masks made compulsory in public in Tehran as COVID toll rises https://t.co/tbBR8ZONQx https://t.co/mk19GoN2xL
U.S. COVID-19 cases hit two-month high, 10 states report record increases https://t.co/JT0Uzv6x7Y https://t.co/QwtlhyxW5H
Nurses suffer burn-out, psychological distress in COVID fight - association https://t.co/DjC8ROUe5E https://t.co/BLI23IWIZL
England boss Southgate fears changes to Euro 2020 format due to COVID-19 https://t.co/rrrjKC

In [12]:
cleaned_text_China = [lemma_pos_cleaner(txt) for txt in text_China]
cleaned_text_USA = [lemma_pos_cleaner(txt) for txt in text_USA]
cleaned_text_China_USA = [lemma_pos_cleaner(txt) for txt in text_China_USA]

print('China:')
print(cleaned_text_China[0:10])
print()
print('USA:')
print(cleaned_text_USA[0:10])
print()
print('China&USA:')
print(cleaned_text_China_USA[0:10])

China:
['talk university challenge china face deal misconception people medium watch', 'traditional medicine use battle covid answer may surprise', 'italy report first death year old man northern new case report', 'evacuee diamond princess cruise ship land hong hong infect', 'fear stop one couple get marry wedding guest include couple wore surgical mask ceremony', 'group molecular biologist might invent portable device detect infect', 'china world make great stride track infectious disease take unexpected turn make hard track', 'member expert advisory committee say important use technology fight', 'world health organization announce lead team international expert currently china', 'outside china confirm case novel covid half cruise ship dock japan rest scatter among country mostly asia']

USA:
['coronavirus bring anti sentiment south korea', 'coronavirus issue could come bad time start line muller boston university', 'claim two life iran', 'happen catch new', 'vaccine available', 'firs

In [13]:
freq_dict_China = frequency_dictionary(cleaned_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  4191
USA:  6005
China&USA:  6923


In [14]:
# Most frequent words
print('China')
print([key for key in freq_dict_China.keys() if freq_dict_China[key]>200])
print()
print('USA')
print([key for key in freq_dict_USA.keys() if freq_dict_USA[key]>400])
print()
print('China&USA')
print([key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]>600])

China
['china', 'covid', 'case', 'vaccine', 'novel', 'coronavirus', 'new', 'say', 'report', 'outbreak', 'hospital', 'health', 'people', 'country', 'patient', 'fight', 'confirm', 'first', 'death', 'president', 'million', 'day', 'world', 'test', 'pandemic', 'city', 'year', 'infection', 'live', 'amid', 'one', 'medical', 'epidemic', 'number', 'receive', 'control', 'hubei']

USA
['covid', 'coronavirus', 'vaccine', 'say', 'case', 'new', 'china', 'trump', 'test', 'president', 'report', 'death', 'people', 'health', 'outbreak', 'pandemic', 'first', 'state', 'positive', 'country', 'million', 'day', 'world', 'spread', 'infection', 'rise', 'virus', 'house', 'johnson', 'get', 'week', 'trial', 'year', 'month', 'one', 'use', 'global', 'late', 'white', 'take', 'two', 'could', 'official', 'record', 'may']

China&USA
['covid', 'coronavirus', 'vaccine', 'say', 'china', 'case', 'new', 'test', 'report', 'trump', 'president', 'health', 'people', 'outbreak', 'death', 'first', 'country', 'pandemic', 'novel', 

# Build Network

In [17]:
df_China = pd.DataFrame.from_dict(freq_dict_China, orient='index').reset_index()
df_China.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China.sort_values(by=['Count'], ascending=False, inplace=True)
df_China.reset_index(inplace=True)
df_China.drop(columns="index",inplace=True)

df_USA = pd.DataFrame.from_dict(freq_dict_USA, orient='index').reset_index()
df_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_USA.reset_index(inplace=True)
df_USA.drop(columns="index",inplace=True)

df_China_USA = pd.DataFrame.from_dict(freq_dict_China_USA, orient='index').reset_index()
df_China_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_China_USA.reset_index(inplace=True)
df_China_USA.drop(columns="index",inplace=True)

print('China')
print(df_China.iloc[0:30])
print()
print('USA')
print(df_USA.iloc[0:30])
print()
print('China&USA')
print(df_China_USA.iloc[0:30])
print()

China
           Word  Count
0         china   1624
1         covid   1622
2          case    951
3       vaccine    928
4         novel    861
5   coronavirus    861
6           new    684
7           say    654
8        report    511
9      outbreak    496
10     hospital    475
11       health    463
12       people    424
13      country    391
14      patient    371
15        fight    358
16      confirm    341
17        first    329
18        death    301
19    president    289
20      million    278
21          day    272
22        world    265
23         test    250
24     pandemic    247
25         city    224
26         year    222
27    infection    221
28         live    219
29         amid    218

USA
           Word  Count
0         covid   7150
1   coronavirus   5053
2       vaccine   3282
3           say   2741
4          case   1943
5           new   1890
6         china   1694
7         trump   1320
8          test   1254
9     president   1150
10       report    973


In [18]:
keys_China = freq_dict_China.keys()  
keys_USA = freq_dict_USA.keys()  
keys_China_USA = freq_dict_China_USA.keys()  

In [19]:
def create_network(cleaned_text):
  network = {}
  #connect the word that appear in the same tweets
  for row in cleaned_text:
    combined_list = [word for word in str.split(row)]
    #for pair in itertools.product(combined_list, combined_list):
    #print(combined_list)
    for pair in itertools.product(combined_list, combined_list):
          #exclude self-loops and count each pair only once because our graph is undirected and we do not take self-loops into account
          if pair[0]!=pair[1] and not(pair[::-1] in network):
              network.setdefault(pair,0)
              network[pair] += 1 
  network_df = pd.DataFrame.from_dict(network, orient="index")
  network_df.columns = ["weight"]
  network_df.sort_values(by="weight",inplace=True, ascending=False)
  return network, network_df

In [20]:
network_China, network_df_China = create_network(cleaned_text_China)
network_USA, network_df_USA = create_network(cleaned_text_USA)
network_China_USA, network_df_China_USA = create_network(cleaned_text_China_USA)

In [21]:
print('China:')
print(network_df_China.iloc[0:30])
print()
print('USA:')
print(network_df_USA.iloc[0:30])
print()
print('China&USA:')
print(network_df_China_USA.iloc[0:30])
print()

China:
                      weight
(vaccine, covid)         582
(new, case)              517
(china, novel)           501
(report, case)           440
(china, covid)           404
(confirm, case)          403
(china, coronavirus)     320
(case, covid)            313
(report, new)            309
(china, outbreak)        302
(novel, coronavirus)     294
(china, vaccine)         293
(china, case)            279
(patient, hospital)      275
(say, china)             258
(death, case)            258
(china, fight)           254
(covid, say)             249
(new, covid)             244
(novel, outbreak)        226
(report, covid)          212
(china, new)             209
(case, coronavirus)      199
(million, covid)         199
(covid, pandemic)        194
(case, number)           190
(covid, country)         190
(case, total)            183
(health, covid)          180
(vaccine, receive)       180

USA:
                         weight
(vaccine, covid)           2180
(say, covid)            

#Graph


In [22]:
def get_graph(network):
  up_weighted = []
  for edge in network:
      #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
      up_weighted.append((edge[0],edge[1],network[edge]))
      
  #print(network)
  #print(up_weighted[0:10])
  G = nx.Graph()
  G.add_weighted_edges_from(up_weighted)
  return G

In [23]:
G_China = get_graph(network_China)
G_USA = get_graph(network_USA)
G_China_USA = get_graph(network_China_USA)

In [24]:
print('China:')
print('Nodes: ',len(G_China.nodes()))
print('Edges: ',len(G_China.edges()))
print('Is connected: ',nx.is_connected(G_China))
print()
print('USA:')
print('Nodes: ',len(G_USA.nodes()))
print('Edges: ',len(G_USA.edges()))
print('Is connected: ',nx.is_connected(G_USA))
print()
print('China&USA:')
print('Nodes: ',len(G_China_USA.nodes()))
print('Edges: ',len(G_China_USA.edges()))
print('Is connected: ',nx.is_connected(G_China_USA))

China:
Nodes:  4191
Edges:  152683
Is connected:  True

USA:
Nodes:  6005
Edges:  303941
Is connected:  True

China&USA:
Nodes:  6923
Edges:  402388
Is connected:  True


# PageRank

In [25]:
# Calculating the pagerank on graph G, teleportation probability here is 0.15 but since the graph is strongly connected we can set it to zero if we want
pr_China = nx.algorithms.pagerank(G_China,alpha = 1)
pr_China = dict(sorted(pr_China.items(), key=lambda item: item[1],reverse  = True))

pr_USA = nx.algorithms.pagerank(G_USA,alpha = 1)
pr_USA = dict(sorted(pr_USA.items(), key=lambda item: item[1],reverse  = True))

pr_China_USA = nx.algorithms.pagerank(G_China_USA,alpha = 1)
pr_China_USA = dict(sorted(pr_China_USA.items(), key=lambda item: item[1],reverse  = True))

In [26]:
def threshold(vector,threshold):

  l = [(el,vector[el]) for el in vector if vector[el] >= threshold ]

  return pd.DataFrame(l)

In [27]:
def threshold_reverse(vector,threshold):

  l = [(el,vector[el]) for el in vector if vector[el] < threshold ]

  return pd.DataFrame(l)

In [28]:
thr = 0.0004
print('China: ', len(threshold(pr_China,thr)))
print()
print(threshold(pr_China,thr).iloc[:30])
print()
print('USA: ', len(threshold(pr_USA,thr)))
print()
print(threshold(pr_USA,thr).iloc[:30])
print()
print('China&USA: ', len(threshold(pr_China_USA,thr)))
print()
print(threshold(pr_China_USA,thr).iloc[:30])

China:  484

              0         1
0         china  0.027595
1         covid  0.025882
2          case  0.015618
3         novel  0.014520
4   coronavirus  0.013722
5       vaccine  0.013662
6           say  0.013023
7           new  0.011256
8        report  0.008870
9        health  0.008859
10     outbreak  0.008342
11       people  0.007991
12     hospital  0.007729
13      country  0.007434
14      patient  0.006174
15      confirm  0.006000
16        fight  0.005848
17        first  0.005668
18        death  0.005319
19          day  0.005225
20    president  0.005152
21        world  0.005126
22      million  0.004771
23         city  0.004512
24     pandemic  0.004429
25         year  0.004421
26         test  0.004327
27          one  0.004159
28       number  0.004075
29      medical  0.004008

USA:  521

              0         1
0         covid  0.035928
1   coronavirus  0.026982
2           say  0.017887
3       vaccine  0.017142
4           new  0.010416
5         tru

# TF-IDF

In [31]:
tfidf = TfidfVectorizer(ngram_range=(1,1))   # ngram range can be changed to obtain measures regarding n grams instead of single words

X_China = tfidf.fit_transform(cleaned_text_China).toarray()    # entry (i,j) if Tfidf measure of word_list[j] in document i
word_list_China = tfidf.get_feature_names_out()

X_USA = tfidf.fit_transform(cleaned_text_USA).toarray()
word_list_USA = tfidf.get_feature_names_out()

X_China_USA = tfidf.fit_transform(cleaned_text_China_USA).toarray()
word_list_China_USA = tfidf.get_feature_names_out()


In [32]:
tfidf_df_China = pd.DataFrame(X_China,columns = word_list_China)

tfidf_df_USA = pd.DataFrame(X_USA,columns = word_list_USA)

tfidf_df_China_USA = pd.DataFrame(X_China_USA,columns = word_list_China_USA)

In [33]:
tfidf_word_measure_China = np.mean(tfidf_df_China,axis = 0)
tfidf_word_measure_China = tfidf_word_measure_China.sort_values(ascending = False)
tfidf_word_measure_USA = np.mean(tfidf_df_USA,axis = 0)
tfidf_word_measure_USA = tfidf_word_measure_USA.sort_values(ascending = False)
tfidf_word_measure_China_USA = np.mean(tfidf_df_China_USA,axis = 0)
tfidf_word_measure_China_USA = tfidf_word_measure_China_USA.sort_values(ascending = False)

In [34]:
print('China:')
print(tfidf_word_measure_China[0:30])
print()
print('USA:')
print(tfidf_word_measure_USA[0:30])
print()
print('China&USA:')
print(tfidf_word_measure_China_USA[0:30])
print()

China:
china          0.041836
covid          0.041499
case           0.033761
vaccine        0.032448
coronavirus    0.028873
novel          0.027566
new            0.027195
report         0.021293
watch          0.020725
say            0.019762
hospital       0.019155
outbreak       0.018205
health         0.016940
confirm        0.015871
patient        0.015501
people         0.015137
fight          0.014955
country        0.014687
first          0.014617
death          0.014212
million        0.013033
live           0.011908
president      0.011829
infection      0.011528
late           0.011439
day            0.011211
receive        0.011106
update         0.010872
test           0.010807
world          0.010605
dtype: float64

USA:
covid          0.053046
coronavirus    0.044115
vaccine        0.035915
case           0.028648
say            0.027138
new            0.026402
china          0.024641
report         0.018183
test           0.017357
death          0.016380
trump       

# reduced graph

In [35]:
# less important words:
less_important_words_China = list(threshold_reverse(pr_China,thr)[0])

less_important_words_USA = list(threshold_reverse(pr_USA,thr)[0])

less_important_words_China_USA = list(threshold_reverse(pr_China_USA,thr)[0])

In [37]:
def clean_words(cleaned_text, unuseful_words):
  re_cleaned_text = cleaned_text.copy()
  for txt in range(len(re_cleaned_text)):
    if txt % 2000 == 0:
      print(txt, '/',len(re_cleaned_text))
    w = re_cleaned_text[txt].split()
    for word in unuseful_words:
      while word in w:
        w.remove(word)
    re_cleaned_text[txt] = ' '.join(w)
  return re_cleaned_text

In [38]:
cleaned_mostimp_text_China = clean_words(cleaned_text_China,less_important_words_China)
cleaned_mostimp_text_USA = clean_words(cleaned_text_USA,less_important_words_USA)
cleaned_mostimp_text_China_USA = clean_words(cleaned_text_China_USA,less_important_words_China_USA)

0 / 5080
2000 / 5080
4000 / 5080
0 / 14553
2000 / 14553
4000 / 14553
6000 / 14553
8000 / 14553
10000 / 14553
12000 / 14553
14000 / 14553
0 / 19633
2000 / 19633
4000 / 19633
6000 / 19633
8000 / 19633
10000 / 19633
12000 / 19633
14000 / 19633
16000 / 19633
18000 / 19633


In [39]:
freq_dict_China = frequency_dictionary(cleaned_mostimp_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_mostimp_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_mostimp_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  484
USA:  521
China&USA:  509


In [40]:
df_China = pd.DataFrame.from_dict(freq_dict_China, orient='index').reset_index()
df_China.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China.sort_values(by=['Count'], ascending=False, inplace=True)
df_China.reset_index(inplace=True)
df_China.drop(columns="index",inplace=True)

df_USA = pd.DataFrame.from_dict(freq_dict_USA, orient='index').reset_index()
df_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_USA.reset_index(inplace=True)
df_USA.drop(columns="index",inplace=True)

df_China_USA = pd.DataFrame.from_dict(freq_dict_China_USA, orient='index').reset_index()
df_China_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_China_USA.reset_index(inplace=True)
df_China_USA.drop(columns="index",inplace=True)

print('China')
print(df_China.iloc[0:30])
print()
print('USA')
print(df_USA.iloc[0:30])
print()
print('China&USA')
print(df_China_USA.iloc[0:30])
print()

China
           Word  Count
0         china   1624
1         covid   1622
2          case    951
3       vaccine    928
4         novel    861
5   coronavirus    861
6           new    684
7           say    654
8        report    511
9      outbreak    496
10     hospital    475
11       health    463
12       people    424
13      country    391
14      patient    371
15        fight    358
16      confirm    341
17        first    329
18        death    301
19    president    289
20      million    278
21          day    272
22        world    265
23         test    250
24     pandemic    247
25         city    224
26         year    222
27    infection    221
28         live    219
29         amid    218

USA
           Word  Count
0         covid   7150
1   coronavirus   5053
2       vaccine   3282
3           say   2741
4          case   1943
5           new   1890
6         china   1694
7         trump   1320
8          test   1254
9     president   1150
10       report    973


In [41]:
keys_China = freq_dict_China.keys()  
keys_USA = freq_dict_USA.keys()  
keys_China_USA = freq_dict_China_USA.keys()  

In [42]:
network_China, network_df_China = create_network(cleaned_mostimp_text_China)
network_USA, network_df_USA = create_network(cleaned_mostimp_text_USA)
network_China_USA, network_df_China_USA = create_network(cleaned_mostimp_text_China_USA)

In [43]:
print('China:')
print(network_df_China.iloc[0:30])
print()
print('USA:')
print(network_df_USA.iloc[0:30])
print()
print('China&USA:')
print(network_df_China_USA.iloc[0:30])
print()

China:
                      weight
(vaccine, covid)         582
(new, case)              517
(china, novel)           501
(report, case)           440
(china, covid)           404
(confirm, case)          403
(china, coronavirus)     320
(case, covid)            313
(report, new)            309
(china, outbreak)        302
(novel, coronavirus)     294
(china, vaccine)         293
(china, case)            279
(patient, hospital)      275
(say, china)             258
(death, case)            258
(china, fight)           254
(covid, say)             249
(new, covid)             244
(novel, outbreak)        226
(report, covid)          212
(china, new)             209
(case, coronavirus)      199
(million, covid)         199
(covid, pandemic)        194
(covid, country)         190
(case, number)           190
(case, total)            183
(vaccine, receive)       180
(health, covid)          180

USA:
                         weight
(vaccine, covid)           2180
(say, covid)            

In [44]:
def get_graph(network):
  up_weighted = []
  for edge in network:
      #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
      up_weighted.append((edge[0],edge[1],network[edge]))
      
  #print(network)
  #print(up_weighted[0:10])
  G = nx.Graph()
  G.add_weighted_edges_from(up_weighted)
  return G

In [45]:
G_China = get_graph(network_China)
G_USA = get_graph(network_USA)
G_China_USA = get_graph(network_China_USA)

In [46]:
print('China:')
print('Nodes: ',len(G_China.nodes()))
print('Edges: ',len(G_China.edges()))
print('Is connected: ',nx.is_connected(G_China))
print()
print('USA:')
print('Nodes: ',len(G_USA.nodes()))
print('Edges: ',len(G_USA.edges()))
print('Is connected: ',nx.is_connected(G_USA))
print()
print('China&USA:')
print('Nodes: ',len(G_China_USA.nodes()))
print('Edges: ',len(G_China_USA.edges()))
print('Is connected: ',nx.is_connected(G_China_USA))

China:
Nodes:  484
Edges:  43085
Is connected:  True

USA:
Nodes:  521
Edges:  70226
Is connected:  True

China&USA:
Nodes:  509
Edges:  78970
Is connected:  True


#Save edge list

In [47]:
filename = './edgelist_China.csv'
nx.write_weighted_edgelist(G_China, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_China.csv
files.download("edgelist_China.csv")

filename = './edgelist_USA.csv'
nx.write_weighted_edgelist(G_USA, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_USA.csv
files.download("edgelist_USA.csv")

filename = './edgelist_China_USA.csv'
nx.write_weighted_edgelist(G_China_USA, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_China_USA.csv
files.download("edgelist_China_USA.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>





# Create Node List




In [48]:
def nodes(freq_dict, name):
  word_nodes = pd.DataFrame.from_dict(freq_dict,orient="index")
  word_nodes.reset_index(inplace=True)
  word_nodes["Label"] = word_nodes["index"]
  word_nodes.rename(columns={"index":"Id",0:"delete"},inplace=True)
  word_nodes = word_nodes.drop(columns=['delete'])
  nodelist = pd.DataFrame()
  nodelist = nodelist.append(word_nodes, ignore_index=True)

  nodelist = nodelist.to_csv("nodelist_"+name+".csv",index=False)
  files.download("nodelist_"+name+".csv")
  return nodelist, word_nodes

In [49]:
nodelist_China, word_nodes_China = nodes(freq_dict_China,"China")
nodelist_USA, word_nodes_USA = nodes(freq_dict_USA,"USA")
nodelist_China_USA, word_nodes_China_USA = nodes(freq_dict_China_USA,"China_USA")

print('China:')
print(word_nodes_China.head())
print()
print('USA:')
print(word_nodes_USA.head())
print()
print('China&USA:')
print(word_nodes_China_USA.head())
print()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

China:
        Id    Label
0    china    china
1    covid    covid
2     case     case
3  vaccine  vaccine
4    novel    novel

USA:
            Id        Label
0        covid        covid
1  coronavirus  coronavirus
2      vaccine      vaccine
3          say          say
4         case         case

China&USA:
            Id        Label
0        covid        covid
1  coronavirus  coronavirus
2      vaccine      vaccine
3          say          say
4        china        china

